In [301]:
df = pd.read_csv("temp_movie_data.csv")

In [12]:
df = pd.read_csv("./data/total_data.csv")

df["장르"][df["장르"].isna()] = "['']"  # 이후에는 제거
df["장르"] = df["장르"].apply(lambda x: eval(x))
df["장르"] = df["장르"].apply(lambda x : (' ').join(x))

isna_index = df["평점"].isna()
df["평점"][isna_index] = "['']"
df["평점"] = df["평점"].apply(lambda x : eval(x))

In [9]:

def print_ls_1(x):
    if len(x) == 1:
        return 0
    else:
        return x[1].split()[0]
    
def print_ls_2(x):
    if len(x) == 1:
        return "0k"
    else:
        try:
            return x[1].split()[1].strip("()")
        except:
            return "0k"
        
df["평가수"] = df["평점"].apply(print_ls_2)
df["평점"] = df["평점"].apply(print_ls_1)
df["평가수"] = df["평가수"].apply(lambda x: str(int(x[:-1]) * 1000) if x[-1] == "m" else x[:-1])
df["평점"] = df["평점"].astype("float")
df["평가수"] = df["평가수"].astype("int")

C = df["평점"].mean()
m = df["평가수"].quantile(0.6)

In [10]:

def weighted_vote_average(record):
    v = record["평가수"]
    R = record["평점"]
    
    return ((v/(v+m)) * R) + ((m/(m+v)) * C)
df["가중치평점"] = df.apply(weighted_vote_average, axis=1)

not_index = df["감독"].notna()
df["감독"][not_index] = df["감독"][not_index].apply(lambda x: x.strip("['']"))
df["감독"][not_index] = df["감독"][not_index].apply(lambda x : x.split(" , "))

def sum_list(x):
    ls = []
    for i in x[:5]:
        ls.append("_".join(i.split(" ")))
    return " ".join(ls)

df["감독"][df["감독"].isna()] = "['']"
# df["감독"] = df["감독"].apply(lambda x: eval(x))
df["감독"] = df["감독"].apply(sum_list)

df["출연진"][df["출연진"].isna()] = "['']"
df["출연진"] = df["출연진"].apply(lambda x: eval(x))
df["출연진"] = df["출연진"].apply(sum_list)

df["특징"] = df["장르"] + " " + df["종류"] + " " + df["감독"] + " " + df["출연진"]


In [11]:
df["특징"]

0        코미디 가족 영화 Chris_Columbus Macaulay_Culkin Joe_P...
1        코미디 Made in Europe 영화 Jeremiah_S._Chechik Chev...
2        판타지 SF 액션 코미디 영화 Ivan_Reitman Bill_Murray Dan_...
3        판타지 액션 코미디 SF 영화 Ivan_Reitman Bill_Murray Dan_...
4        판타지 드라마 가족 영화 Frank_Capra James_Stewart Donna_...
                               ...                        
19683                 RealityTV tv프로그램 [ ' ' ] Karan_Johar
19684    RealityTV 음악 tv프로그램 Takao_Kinoshita_',_'_Manab...
19685                            RealityTV tv프로그램 [ ' ' ] 
19686    RealityTV tv프로그램 [ ' ' ] Jang_Sung-kyu Yoo_Se-...
19687    SF tv프로그램 [ ' ' ] Chihira_Mochida Aya_Akiyoshi...
Name: 특징, Length: 19688, dtype: object

In [300]:
# title_list = ["나 홀로 집에", "해리 포터와 마법사의 돌", "쇼생크 탈출"]
title_list = titles
sum_find = " ".join(list(df[df["제목"].apply(lambda x : x in title_list)]["특징"]))

title_indexes = []
for title_name in title_list:
    title_movie = df[df["제목"] == title_name]
    title_indexes += list(title_movie.index.values)

NameError: name 'titles' is not defined

In [ ]:
count_vect = CountVectorizer(min_df=0, ngram_range=(1, 2))
genre_mat = count_vect.fit_transform(list(df["특징"]) + [sum_find])
genre_sim = cosine_similarity(genre_mat, genre_mat)
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]

In [279]:

def recommendations(titles, top_n = 20):

    df = read_df()
    # title_list = ["나 홀로 집에", "해리 포터와 마법사의 돌", "쇼생크 탈출"]
    title_list = titles
    sum_find = " ".join(list(df[df["제목"].apply(lambda x : x in title_list)]["특징"]))

    title_indexes = []
    for title_name in title_list:
        title_movie = df[df["제목"] == title_name]
        title_indexes += list(title_movie.index.values)

    count_vect = CountVectorizer(min_df=0, ngram_range=(1, 2))
    genre_mat = count_vect.fit_transform(list(df["특징"]) + [sum_find])
    genre_sim = cosine_similarity(genre_mat, genre_mat)
    genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]
    
    last_index = len(df)
    similar_indexes = genre_sim_sorted_ind[last_index, :(top_n * 2)]
    similar_indexes = similar_indexes.reshape(-1)
    similar_indexes = similar_indexes[similar_indexes != last_index]  # 타이틀 기준 제거
    for i in title_indexes:
        similar_indexes = similar_indexes[similar_indexes != i]  # 타이틀 기준 제거
    similar_indexes
    
    return df.iloc[similar_indexes].sort_values("가중치평점", ascending=False)[:top_n]

In [ ]:
recommendations()

In [258]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

## 데이터 읽기

def read_df():
    df = pd.read_csv("temp_movie_data.csv")

    df["장르"][df["장르"].isna()] = "['']"  # 이후에는 제거
    df["장르"] = df["장르"].apply(lambda x: eval(x))
    df["장르"] = df["장르"].apply(lambda x : (' ').join(x))

    isna_index = df["평점"].isna()
    df["평점"][isna_index] = "['']"
    df["평점"] = df["평점"].apply(lambda x : eval(x))

    def print_ls_1(x):
        if len(x) == 1:
            return 0
        else:
            return x[1].split()[0]
        
    def print_ls_2(x):
        if len(x) == 1:
            return "0k"
        else:
            try:
                return x[1].split()[1].strip("()")
            except:
                return "0k"
            
    df["평가수"] = df["평점"].apply(print_ls_2)
    df["평점"] = df["평점"].apply(print_ls_1)
    df["평가수"] = df["평가수"].apply(lambda x: str(int(x[:-1]) * 1000) if x[-1] == "m" else x[:-1])
    df["평점"] = df["평점"].astype("float")
    df["평가수"] = df["평가수"].astype("int")

    C = df["평점"].mean()
    m = df["평가수"].quantile(0.6)

    def weighted_vote_average(record):
        v = record["평가수"]
        R = record["평점"]
        
        return ((v/(v+m)) * R) + ((m/(m+v)) * C)
    df["가중치평점"] = df.apply(weighted_vote_average, axis=1)

    not_index = df["감독"].notna()
    df["감독"][not_index] = df["감독"][not_index].apply(lambda x: x.strip("['']"))
    df["감독"][not_index] = df["감독"][not_index].apply(lambda x : x.split(" , "))

    def sum_list(x):
        ls = []
        for i in x[:5]:
            ls.append("_".join(i.split(" ")))
        return " ".join(ls)

    df["감독"][df["감독"].isna()] = "['']"
    # df["감독"] = df["감독"].apply(lambda x: eval(x))
    df["감독"] = df["감독"].apply(sum_list)

    df["출연진"][df["출연진"].isna()] = "['']"
    df["출연진"] = df["출연진"].apply(lambda x: eval(x))
    df["출연진"] = df["출연진"].apply(sum_list)

    df["특징"] = df["장르"] + " " + df["감독"] + " " + df["출연진"]
    return df

def recommendations(titles, top_n = 20):

    df = read_df()
    # title_list = ["나 홀로 집에", "해리 포터와 마법사의 돌", "쇼생크 탈출"]
    title_list = titles
    sum_find = " ".join(list(df[df["제목"].apply(lambda x : x in title_list)]["특징"]))

    title_indexes = []
    for title_name in title_list:
        title_movie = df[df["제목"] == title_name]
        title_indexes += list(title_movie.index.values)

    count_vect = CountVectorizer(min_df=0, ngram_range=(1, 2))
    genre_mat = count_vect.fit_transform(list(df["특징"]) + [sum_find])
    genre_sim = cosine_similarity(genre_mat, genre_mat)
    genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]
    
    last_index = len(df)
    similar_indexes = genre_sim_sorted_ind[last_index, :(top_n * 2)]
    similar_indexes = similar_indexes.reshape(-1)
    similar_indexes = similar_indexes[similar_indexes != last_index]  # 타이틀 기준 제거
    for i in title_indexes:
        similar_indexes = similar_indexes[similar_indexes != i]  # 타이틀 기준 제거
    similar_indexes
    
    return df.iloc[similar_indexes].sort_values("가중치평점", ascending=False)[:top_n]

# TEST

In [257]:
result = recommendations(["나 홀로 집에", "해리 포터와 마법사의 돌", "쇼생크 탈출"])
result[["제목", "감독", "장르", "출연진", "평점", "가중치평점", "이미지"]]

,제목,감독,장르,출연진,평점,가중치평점,이미지
3401,라따뚜이,Brad_Bird,애니메이션 판타지 코미디 가족 드라마,Patton_Oswalt Ian_Holm Lou_Romano Brian_Denneh...,8.0,7.900844,https://images.justwatch.com/poster/244791829/...
30,팀버튼의 크리스마스 악몽,Henry_Selick,판타지 애니메이션 가족,Chris_Sarandon Danny_Elfman Catherine_O'Hara K...,8.0,7.803053,https://images.justwatch.com/poster/14855415/s...
1806,해리 포터와 아즈카반의 죄수,Alfonso_Cuarón,판타지 가족,Daniel_Radcliffe Rupert_Grint Emma_Watson Juli...,7.9,7.793852,https://images.justwatch.com/poster/166638550/...
3392,해리 포터와 불의 잔,Mike_Newell,판타지 가족,Daniel_Radcliffe Rupert_Grint Emma_Watson Robb...,7.7,7.606552,https://images.justwatch.com/poster/243802693/...
3495,해리 포터와 혼혈 왕자,David_Yates,판타지 가족 액션,Daniel_Radcliffe Rupert_Grint Emma_Watson Hele...,7.6,7.501467,https://images.justwatch.com/poster/177293523/...
3539,쿵푸팬더,Mark_Osborne John_Stevenson,코미디 가족 판타지 애니메이션 액션 드라마,Jack_Black Dustin_Hoffman Angelina_Jolie Jacki...,7.6,7.488288,https://images.justwatch.com/poster/186677557/...
3530,해리 포터와 불사조 기사단,David_Yates,스릴러 액션 판타지 가족,Daniel_Radcliffe Rupert_Grint Emma_Watson Hele...,7.5,7.414835,https://images.justwatch.com/poster/243842469/...
1815,해리 포터와 비밀의 방,Chris_Columbus,판타지 액션 가족,Daniel_Radcliffe Rupert_Grint Emma_Watson Kenn...,7.4,7.328044,https://images.justwatch.com/poster/177292841/...
510,붉은 돼지,Hayao_Miyazaki,애니메이션 액션 가족 코미디 판타지 로맨스,Shûichirô_Moriyama Tokiko_Katô Sanshi_Katsura ...,7.7,7.238647,https://images.justwatch.com/poster/186959346/...
137,미세스 다웃파이어,Chris_Columbus,드라마 코미디 가족,Robin_Williams Sally_Field Pierce_Brosnan Lisa...,7.0,6.905027,https://images.justwatch.com/poster/46922266/s...


In [282]:
import Recommendation

Recommendation.read_df()

,Unnamed: 0,제목,원제,개봉일,스트리밍,대여,구매,평점,장르,재생 시간,감독,출연진,시놉시스,이미지,평가수,가중치평점,특징
0,0,나 홀로 집에,Home Alone,(1990),"{'Disney Plus': ['정액제', '4K']}",NaN,"{'wavve': ['₩5,500', 'HD'], 'Naver Store': ['₩...",7.6,코미디 가족,['1시간 43분'],Chris_Columbus,Macaulay_Culkin Joe_Pesci Daniel_Stern John_He...,크리스마스 시즌의 시카고. 말썽꾸러기 케빈(맥컬리 컬킨)은 자기의 치즈 피자를 먹은...,https://images.justwatch.com/poster/240463286/...,511,7.502000,코미디 가족 Chris_Columbus Macaulay_Culkin Joe_Pesc...
1,1,크리스마스 대소동,National Lampoon's Christmas Vacation,(1989),NaN,"{'Naver Store': ['₩1,300', 'HD']}","{'Naver Store': ['₩4,950', 'HD']}",7.6,코미디 Made in Europe,['1시간 37분'],Jeremiah_S._Chechik,Chevy_Chase Beverly_D'Angelo Juliette_Lewis Jo...,"클라크 오스왈드(Clark Wilhelm Griswold, Jr.: 체비 체이스 분...",https://images.justwatch.com/poster/248460937/...,175,7.350955,코미디 Made in Europe Jeremiah_S._Chechik Chevy_C...
2,2,고스트 버스터즈,Ghostbusters,(1984),"{'Netflix': ['정액제', '4K']}","{'Naver Store': ['₩1,300', 'HD']}","{'Naver Store': ['₩5,000', 'HD']}",7.8,판타지 SF 액션 코미디,['1시간 41분'],Ivan_Reitman,Bill_Murray Dan_Aykroyd Sigourney_Weaver Harol...,초심리를 연구하는 뉴욕의 괴짜교수 피터 밴크맨(빌 머레이)와 레이몬드 스탠드(댄 애...,https://images.justwatch.com/poster/19416696/s...,373,7.648818,판타지 SF 액션 코미디 Ivan_Reitman Bill_Murray Dan_Ayk...
3,3,고스트 버스터즈 2,Ghostbusters II,(1989),"{'wavve': ['정액제', 'HD'], 'Watcha': ['정액제', 'HD']}","{'wavve': ['₩1,300', 'HD'], 'Naver Store': ['₩...","{'Naver Store': ['₩5,000', 'HD'], 'wavve': ['₩...",6.6,판타지 액션 코미디 SF,['1시간 48분'],Ivan_Reitman,Bill_Murray Dan_Aykroyd Sigourney_Weaver Harol...,고스트 버스터즈 4인방은 악마 '고저'로부터 세계를 구한 후 순탄한 생활을 하지 못...,https://images.justwatch.com/poster/20537077/s...,196,6.552754,판타지 액션 코미디 SF Ivan_Reitman Bill_Murray Dan_Ayk...
4,4,멋진 인생,It's a Wonderful Life,(1946),"{'Plex': ['무료', 'HD']}",NaN,"{'Naver Store': ['₩1,000']}",8.6,판타지 드라마 가족,['2시간 10분'],Frank_Capra,James_Stewart Donna_Reed Lionel_Barrymore Thom...,조지 베일리는 베드폴드 마을 사람들을 위해 평생을 봉사하며 살아온 인물. 그는 항상...,https://images.justwatch.com/poster/244992618/...,429,8.393873,판타지 드라마 가족 Frank_Capra James_Stewart Donna_Ree...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,3995,인터내셔널,The International,(2009),"{'Netflix': ['정액제', 'HD']}","{'wavve': ['₩1,300', 'HD']}","{'wavve': ['₩5,500', 'HD']}",6.5,스릴러 범죄 드라마 액션 Made in Europe,['1시간 58분'],Tom_Tykwer,Clive_Owen Naomi_Watts Armin_Mueller-Stahl Ulr...,누구도 통제할 수 없는 음모...,https://images.justwatch.com/poster/247510757/...,94,6.448965,스릴러 범죄 드라마 액션 Made in Europe Tom_Tykwer Clive_...
3996,3996,더 시크릿,Si j'étais toi,(2007),"{'wavve': ['정액제', 'HD'], 'Watcha': ['정액제', 'HD']}","{'wavve': ['₩1,200', 'HD']}","{'Naver Store': ['₩1,200', 'HD']}",6.3,드라마 판타지 공포 로맨스 스릴러 Made in Europe,['1시간 32분'],Vincent_Perez,David_Duchovny Olivia_Thirlby Lili_Taylor Mill...,"Husband, wife, and daughter have moved from Bo...",https://images.justwatch.com/poster/174366795/...,3,6.334961,드라마 판타지 공포 로맨스 스릴러 Made in Europe Vincent_Pere...
3997,3997,옐라,Yella,(2007),"{'Mubi': ['정액제', 'HD']}","{'wavve': ['₩7,000', 'HD']}",NaN,6.7,스릴러 Made in Europe 드라마 공포 로맨스,['1시간 29분'],Christian_Petzold,Nina_Hoss Devid_Striesow Hinnerk_Schönemann Bu...,옐라는 실패한 결혼생활과 부서진 꿈을 뒤로한 채 새로운 삶을 찾아 고향을 떠나기로 ...,https://images.justwatch.com/poster/8797266/s2...,3,6.361048,스릴러 Made in Europe 드라마 공포 로맨스 Christian_Petzol...
3998,3998,카운터페이터,Die Fälscher,(2007),"{'Watcha': ['정액제', 'HD']}",NaN,"{'Naver Store': ['₩1,200', 'HD']}",7.6,드라마 Made in Europe 역사 전쟁 범죄,['1시간 38분'],Stefan_Ruzowitzky,Karl_Markovics August_Diehl Devid_Striesow Mar...,역사상 최대의 위조지폐 작전에 투입된 천재적인 위조 전문가! 세상조차 속일 수 있었...,https://images.justwatch.com/poster/199925259/...,44,6.975956,드라마 Made in Europe 역사 전쟁 범죄 Stefan_Ruzowitzky ...


In [281]:
import Recommendation

result = Recommendation.recommendations(["나 홀로 집에", "해리 포터와 마법사의 돌", "쇼생크 탈출"])
result

,Unnamed: 0,제목,원제,개봉일,스트리밍,대여,구매,평점,장르,재생 시간,감독,출연진,시놉시스,이미지,평가수,가중치평점,특징
3401,3401,라따뚜이,Ratatouille,(2007),"{'Disney Plus': ['정액제', '4K']}",NaN,"{'wavve': ['₩7,150', 'HD'], 'Naver Store': ['₩...",8.0,애니메이션 판타지 코미디 가족 드라마,['1시간 51분'],Brad_Bird,Patton_Oswalt Ian_Holm Lou_Romano Brian_Denneh...,"절대미각, 빠른 손놀림, 끓어 넘치는 열정의 소유자 레미. 프랑스 최고의 요리사를 ...",https://images.justwatch.com/poster/244791829/...,678,7.900844,애니메이션 판타지 코미디 가족 드라마 Brad_Bird Patton_Oswalt I...
30,30,팀버튼의 크리스마스 악몽,The Nightmare Before Christmas,(1993),"{'Disney Plus': ['정액제', 'HD']}",NaN,"{'Naver Store': ['₩5,500', 'HD']}",8.0,판타지 애니메이션 가족,['1시간 16분'],Henry_Selick,Chris_Sarandon Danny_Elfman Catherine_O'Hara K...,1년에 한 번 할로윈 축제를 위해서 사는 할로윈 마을의 리더인 잭은 해골 귀신이다....,https://images.justwatch.com/poster/14855415/s...,320,7.803053,판타지 애니메이션 가족 Henry_Selick Chris_Sarandon Danny...
1806,1806,해리 포터와 아즈카반의 죄수,Harry Potter and the Prisoner of Azkaban,(2004),"{'wavve': ['정액제', 'HD']}","{'Naver Store': ['₩1,300', 'HD'], 'wavve': ['₩...","{'wavve': ['₩4,950', 'HD'], 'Naver Store': ['₩...",7.9,판타지 가족,['2시간 21분'],Alfonso_Cuarón,Daniel_Radcliffe Rupert_Grint Emma_Watson Juli...,13세가 된 해리 포터는 또 한번의 여름 방학을 이모 가족인 더즐리 일가와 우울하게...,https://images.justwatch.com/poster/166638550/...,590,7.793852,판타지 가족 Alfonso_Cuarón Daniel_Radcliffe Rupert_...
3392,3392,해리 포터와 불의 잔,Harry Potter and the Goblet of Fire,(2005),"{'wavve': ['정액제', 'HD']}","{'Naver Store': ['₩1,300', 'HD'], 'wavve': ['₩...","{'wavve': ['₩4,950', 'HD'], 'Naver Store': ['₩...",7.7,판타지 가족,['2시간 37분'],Mike_Newell,Daniel_Radcliffe Rupert_Grint Emma_Watson Robb...,요즘 들어 매일 꾸는 악몽 때문에 이마의 상처에 더욱 통증을 느끼는 해리는 친구 론...,https://images.justwatch.com/poster/243802693/...,584,7.606552,판타지 가족 Mike_Newell Daniel_Radcliffe Rupert_Gri...
3495,3495,해리 포터와 혼혈 왕자,Harry Potter and the Half-Blood Prince,(2009),"{'wavve': ['정액제', 'HD']}","{'Naver Store': ['₩1,300', 'HD'], 'wavve': ['₩...","{'wavve': ['₩4,950', 'HD'], 'Naver Store': ['₩...",7.6,판타지 가족 액션,['2시간 33분'],David_Yates,Daniel_Radcliffe Rupert_Grint Emma_Watson Hele...,어둠의 세력이 더욱 강력해져 머글 세계와 호그와트까지 위협해온다. 위험한 기운을 감...,https://images.justwatch.com/poster/177293523/...,508,7.501467,판타지 가족 액션 David_Yates Daniel_Radcliffe Rupert_...
3539,3539,쿵푸팬더,Kung Fu Panda,(2008),"{'Netflix': ['정액제', 'HD'], 'wavve': ['정액제', 'H...","{'wavve': ['₩1,200', 'HD'], 'Naver Store': ['₩...","{'wavve': ['₩4,000', 'HD'], 'Naver Store': ['₩...",7.6,코미디 가족 판타지 애니메이션 액션 드라마,['1시간 30분'],Mark_Osborne John_Stevenson,Jack_Black Dustin_Hoffman Angelina_Jolie Jacki...,평화의 계곡에서 아버지의 국수 가게를 돕고 있는 팬더 포. 아버지는 국수의 비법을 ...,https://images.justwatch.com/poster/186677557/...,443,7.488288,코미디 가족 판타지 애니메이션 액션 드라마 Mark_Osborne John_Stev...
3530,3530,해리 포터와 불사조 기사단,Harry Potter and the Order of the Phoenix,(2007),"{'wavve': ['정액제', 'HD']}","{'Naver Store': ['₩1,300', 'HD'], 'wavve': ['₩...","{'wavve': ['₩4,950', 'HD'], 'Naver Store': ['₩...",7.5,스릴러 액션 판타지 가족,['2시간 18분'],David_Yates,Daniel_Radcliffe Rupert_Grint Emma_Watson Hele...,길고도 지루한 여름 날 호그와트 마법학교 다섯 번째 해를 기다리고 있는 해리포터(다...,https://images.justwatch.com/poster/243842469/...,544,7.414835,스릴러 액션 판타지 가족 David_Yates Daniel_Radcliffe Rup...
1815,1815,해리 포터와 비밀의 방,Harry Potter and the Chamber of Secrets,(2002),"{'wavve': ['정액제', 'HD']}","{'Naver Store': ['₩1,300', 'HD'], 'wavve': ['₩...","{'wavve': ['₩4,950', 'HD'], 'Naver Store': ['₩...",7.4,판타지 액션 가족,['2시간 41분'],Chris_Columbus,Daniel_Radcliffe Rupert_Grint Emma_Watson Kenn...,해리 포터에겐 이번 여름방학이 별로 즐겁질 못했다. 마법이라면 질색을 하는 페투니아...,https://images.justwatch.com/poster/177292841/...,592,7.328044,판타지 액션 가족 Chris_Columbus Daniel_Radcliffe Rupe...
510,510,붉은 돼지,紅の豚,(1992),"{'Netflix': ['정액제', 'HD']}",NaN,NaN,7.7,애니메이션 액션 가족 코미디 판타지 로맨스,['1시간 34분'],Hayao_Miyazaki,Shûichirô_Moriyama Tokiko_Katô Sanshi_Katsura ...,"제1차 세계대전이 끝난 후, 전쟁의 기억을 지우기 위해 스스로 돼지로 변한 이탈리아...",https://images.justwatch.com/poster/186959346/...,84,7.238647,애니메이션 액션 가족 코미디 판타지 로맨스 Hayao_Miyazaki Shûichi...
137,137,미세스 다웃파이어,Mrs. Doubtfire,(1993),"{'Disney Plus': ['정액제', 'HD']}",NaN,"{'wavve